In [ ]:
import wave
import librosa
import soundfile as sf
from pydub import AudioSegment

In [ ]:
windows_path = "D:/Desktop/coqui_tts/"
root_path = "/home/pnx/Desktop/paddle/"
path = root_path
file_generation = path + "output/" + "2025-03-31-15_52_01.wav"
file_reference = path + "data/" + "2.wav"

In [7]:
def get_sampling_rate_wav(file_path):
    with wave.open(file_path, 'rb') as wav_file:
        # 获取采样率
        sampling_rate = wav_file.getframerate()
        return sampling_rate


def resample_audio(input_path, output_path, target_sr=16000):
    # 加载音频文件，sr=None 表示加载时不进行采样率重采样
    y, sr = librosa.load(input_path, sr=None)

    if sr != target_sr:
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

        sf.write(output_path, y_resampled, target_sr)
        print(f"音频 {input_path} 已重采样为 {target_sr}Hz，并保存为 {output_path}")
    else:
        # 如果采样率已经是目标采样率，直接保存
        sf.write(output_path, y, sr)
        print(f"音频 {input_path} 已是目标采样率 {target_sr}Hz，无需转换")

### 获取采样率数据

In [11]:
sampling_rate = get_sampling_rate_wav(file_reference)
print(f"Sampling rate: {sampling_rate} Hz")

Sampling rate: 16000 Hz


In [27]:
sample_generation = file_generation.replace("output/", "output/resample/")
sample_reference = file_reference.replace("data/", "output/resample/")
resample_audio(file_generation, sample_generation)
resample_audio(file_reference, sample_reference)

音频 D:/Desktop/coqui_tts/output/2025-03-31-15_52_01.wav 已重采样为 16000Hz，并保存为 D:/Desktop/coqui_tts/output/resample/2025-03-31-15_52_01.wav
音频 D:/Desktop/coqui_tts/data/2.wav 已是目标采样率 16000Hz，无需转换


In [26]:
print(sample_reference)

D:/Desktop/coqui_tts/output/resample/2.wav
